In [1]:
import os

# Get the directory containing the current notebook
notebook_dir = os.path.dirname(os.path.abspath("__file__"))

# Change working directory to notebook folder
os.chdir(notebook_dir)

# Verify
print("Current working directory:", os.getcwd())

Current working directory: /home/j/jl1416/sta521/521PredictionProject-1


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import TargetEncoder  # sklearn's TargetEncoder
from xgboost import XGBClassifier
import optuna

# -----------------------------
# Load cleaned data
# -----------------------------
train = pd.read_csv('data/data_cleaned/train_clean.csv')
holdout = pd.read_csv('data/data_cleaned/holdout_cleaned.csv')

y_train = train['damage_grade']
X_train = train.drop(columns=['damage_grade'])

y_holdout = holdout['damage_grade']
X_holdout = holdout.drop(columns=['damage_grade'])

# Features to target‐encode
geo_target = ['geo__geo_level_2_id', 'geo__geo_level_3_id']

In [ ]:
# -----------------------------
# Define Optuna objective
# -----------------------------
def objective(trial):
    # Hyperparameters for XGBoost
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 5),
        'eval_metric': 'mlogloss',
        'random_state': 42
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = []

    for train_idx, val_idx in skf.split(X_train, y_train):
        X_tr = X_train.iloc[train_idx].copy()
        X_val = X_train.iloc[val_idx].copy()
        y_tr = y_train.iloc[train_idx]
        y_val = y_train.iloc[val_idx]

        # Use sklearn TargetEncoder
        te = TargetEncoder(cv=5, shuffle=True, random_state=42, target_type = "multiclass")
        # Note: fit_transform does *cross‑fitting* internally in sklearn version.
        X_tr_enc = te.fit_transform(X_tr[geo_target], y_tr)
        X_val_enc = te.transform(X_val[geo_target])

        # Replace original geo columns with encoded ones
        X_tr_enc = pd.DataFrame(X_tr_enc, index=X_tr.index, columns=te.get_feature_names_out(geo_target))
        X_val_enc = pd.DataFrame(X_val_enc, index=X_val.index, columns=te.get_feature_names_out(geo_target))

        X_tr_full = X_tr.copy()
        X_val_full = X_val.copy()
        X_tr_full.drop(columns=geo_target, inplace=True)
        X_val_full.drop(columns=geo_target, inplace=True)
        # concat encoded
        X_tr_full = pd.concat([X_tr_full, X_tr_enc], axis=1)
        X_val_full = pd.concat([X_val_full, X_val_enc], axis=1)

        # Train model
        model = XGBClassifier(**params,
                              n_jobs=1,)
        model.fit(X_tr_full, y_tr)

        acc = model.score(X_val_full, y_val)
        cv_scores.append(acc)

    # Return mean accuracy so Optuna maximizes it
    return np.mean(cv_scores)

# -----------------------------
# Run Optuna
# -----------------------------
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=25, show_progress_bar=True,)

print("Best params:", study.best_params)
print("Best CV accuracy:", study.best_value)

[I 2025-11-21 17:05:07,097] A new study created in memory with name: no-name-1cd0303a-8686-4326-9ed8-0dc4fc371fbc


  0%|          | 0/25 [00:00<?, ?it/s]

/home/j/jl1416/sta521_project_env/lib64/python3.13/site-packages/xgboost/training.py:199: UserWarning: [17:05:09] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/j/jl1416/sta521_project_env/lib64/python3.13/site-packages/xgboost/training.py:199: UserWarning: [17:05:19] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/j/jl1416/sta521_project_env/lib64/python3.13/site-packages/xgboost/training.py:199: UserWarning: [17:05:30] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/j/jl1416/sta521_project_env/lib64/python3.13/site-packages/xgboost/training.py:199: UserWarning: [17:05:41] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[I 2025-11-21 17:06:03,305] Trial 0 finished with value: 0.741488888888889 and parameters: {'n_estimators': 216, 'max_depth': 3, 'learning_rate': 0.1797943611667039, 'subsample': 0.8048666776835782, 'colsample_bytree': 0.9902810568124285, 'gamma': 0.34000616371133163, 'reg_alpha': 1.1344582316681995, 'reg_lambda': 4.12774455480776}. Best is trial 0 with value: 0.741488888888889.


/home/j/jl1416/sta521_project_env/lib64/python3.13/site-packages/xgboost/training.py:199: UserWarning: [17:06:04] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/j/jl1416/sta521_project_env/lib64/python3.13/site-packages/xgboost/training.py:199: UserWarning: [17:06:13] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/j/jl1416/sta521_project_env/lib64/python3.13/site-packages/xgboost/training.py:199: UserWarning: [17:06:22] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/j/jl1416/sta521_project_env/lib64/python3.13/site-packages/xgboost/training.py:199: UserWarning: [17:06:31] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


...

/home/j/jl1416/sta521_project_env/lib64/python3.13/site-packages/xgboost/training.py:199: UserWarning: [18:23:51] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/j/jl1416/sta521_project_env/lib64/python3.13/site-packages/xgboost/training.py:199: UserWarning: [18:25:05] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/j/jl1416/sta521_project_env/lib64/python3.13/site-packages/xgboost/training.py:199: UserWarning: [18:26:20] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/j/jl1416/sta521_project_env/lib64/python3.13/site-packages/xgboost/training.py:199: UserWarning: [18:27:36] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[I 2025-11-21 18:30:06,055] Trial 23 finished with value: 0.7443888888888889 and parameters: {'n_estimators': 899, 'max_depth': 8, 'learning_rate': 0.01843040760076561, 'subsample': 0.7580526473906376, 'colsample_bytree': 0.5002508793698, 'gamma': 0.053848559715546473, 'reg_alpha': 1.7481508446411889, 'reg_lambda': 3.6386748124193686}. Best is trial 18 with value: 0.7451722222222223.


/home/j/jl1416/sta521_project_env/lib64/python3.13/site-packages/xgboost/training.py:199: UserWarning: [18:30:06] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/j/jl1416/sta521_project_env/lib64/python3.13/site-packages/xgboost/training.py:199: UserWarning: [18:31:03] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/j/jl1416/sta521_project_env/lib64/python3.13/site-packages/xgboost/training.py:199: UserWarning: [18:32:00] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/j/jl1416/sta521_project_env/lib64/python3.13/site-packages/xgboost/training.py:199: UserWarning: [18:32:56] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[I 2025-11-21 18:34:49,474] Trial 24 finished with value: 0.7453555555555555 and parameters: {'n_estimators': 990, 'max_depth': 10, 'learning_rate': 0.04321824739429615, 'subsample': 0.875086850408032, 'colsample_bytree': 0.6474727673680003, 'gamma': 0.8126279635172744, 'reg_alpha': 0.8280296681292646, 'reg_lambda': 4.603013353075994}. Best is trial 24 with value: 0.7453555555555555.
Best params: {'n_estimators': 990, 'max_depth': 10, 'learning_rate': 0.04321824739429615, 'subsample': 0.875086850408032, 'colsample_bytree': 0.6474727673680003, 'gamma': 0.8126279635172744, 'reg_alpha': 0.8280296681292646, 'reg_lambda': 4.603013353075994}
Best CV accuracy: 0.7453555555555555


/home/j/jl1416/sta521_project_env/lib64/python3.13/site-packages/xgboost/training.py:199: UserWarning: [18:34:50] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [ ]:
# -----------------------------
# Train final model on full train set with target encoding
# -----------------------------
te_final = TargetEncoder(cv=5, shuffle=True, random_state=42)
X_train_enc = te_final.fit_transform(X_train[geo_target], y_train)
X_holdout_enc = te_final.transform(X_holdout[geo_target])

X_train_enc = pd.DataFrame(X_train_enc, index=X_train.index, columns=te_final.get_feature_names_out(geo_target))
X_holdout_enc = pd.DataFrame(X_holdout_enc, index=X_holdout.index, columns=te_final.get_feature_names_out(geo_target))

X_train_full = X_train.copy()
X_holdout_full = X_holdout.copy()
X_train_full.drop(columns=geo_target, inplace=True)
X_holdout_full.drop(columns=geo_target, inplace=True)

X_train_full = pd.concat([X_train_full, X_train_enc], axis=1)
X_holdout_full = pd.concat([X_holdout_full, X_holdout_enc], axis=1)

final_model = XGBClassifier(**study.best_params, eval_metric='mlogloss', random_state=42, n_jobs=1)
final_model.fit(X_train_full, y_train)

# -----------------------------
# Evaluate on holdout
# -----------------------------
holdout_acc = final_model.score(X_holdout_full, y_holdout)
print("Holdout accuracy:", holdout_acc)


/home/j/jl1416/sta521_project_env/lib64/python3.13/site-packages/xgboost/core.py:771: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)


Holdout accuracy: 0.74925


In [12]:
import pickle

with open("artifacts/final_model_xgb.pkl", "wb") as f:
    pickle.dump(final_model, f)